# Progetto Computer Vision - Prisco

In [1]:
from IPython.display import display, Markdown

# Crea una tabella markdown per mostrare le metriche
table_md = """
# **Metriche Obiettivo Finali**

| **Metrica**    | **VIDEO 5** | **VIDEO 6** | **MEDIA**     |
|----------------|-------------|-------------|---------------|
| **MSE**        | 0.03840     | 0.00225     | 0.02035       |
"""

# Mostra la tabella in Colab
display(Markdown(table_md))



# **Metriche Obiettivo Finali**

| **Metrica**    | **VIDEO 5** | **VIDEO 6** | **MEDIA**     |
|----------------|-------------|-------------|---------------|
| **MSE**        | 0.03840     | 0.00225     | 0.02035       |




## Modello scelto:
   ### YOLO11
![](https://cdn.prod.website-files.com/6479eab6eb2ed5e597810e9e/66f680814693dd5c3b60dfcb_YOLO11_Thumbnail.png)

## Operazioni preliminari e installazione

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

Si importano le classi di utilità per calcolare le metriche

In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

plt.rcParams["figure.figsize"] = (10, 8)

In [2]:
class TrackingEvaluator:
    def __init__(self, gt_ann_file, pred_file):
        self.gt_ann_file = gt_ann_file
        self.pred_file = pred_file
        self.gt_data_points = {}
        self.pred_points = {}
        self.frames_idx = None

    @staticmethod
    def extract_points_data(xml_content):
        root = ET.fromstring(xml_content)
        points_data = {}

        for track in root.findall(".//track"):
            for point in track.findall("points"):
                data = {
                    'frame': int(point.get("frame")),
                    'outside': int(point.get("outside")),
                    'occluded': int(point.get("occluded")),
                    'keyframe': int(point.get("keyframe")),
                    'points': tuple(map(float, point.get("points").split(","))),
                    'z_order': int(point.get("z_order")),
                }
                if data['frame'] in points_data:
                    print(f'Alert: multiple frame entries for ID {data["frame"]}')
                points_data[data['frame']] = data

        return points_data

    @staticmethod
    def _convert_key(k):
        return int(Path(k).stem)

    def load_data(self):
        # Load ground truth data
        gt_content = Path(self.gt_ann_file).read_text()
        self.gt_data_points = self.extract_points_data(gt_content)

        # Load prediction data
        pred_content = Path(self.pred_file).read_text().replace('-Infinity', '-1').replace('Infinity', '-1')
        raw_pred_points = json.loads(pred_content)
        self.pred_points = {
            self._convert_key(k): v for k, v in raw_pred_points.items() if v['x'] >= 0
        }

    def compute_frame_indices(self):
        ordered_list_pred_frame = sorted(self.pred_points.keys())
        ordered_list_gt_frame = sorted(self.gt_data_points.keys())

        print(f'GT   frames: {ordered_list_gt_frame[0]} - {ordered_list_gt_frame[-1]}')
        print(f'PRED frames: {ordered_list_pred_frame[0]} - {ordered_list_pred_frame[-1]}')

        self.frames_idx = (
            min(ordered_list_gt_frame[0], ordered_list_pred_frame[0]),
            max(ordered_list_gt_frame[-1], ordered_list_pred_frame[-1]),
        )
        print(f'Frame Index Range: {self.frames_idx}')

    @staticmethod
    def is_match(x1, y1, x2, y2, threshold=4):
        p1 = np.array((x1, y1))
        p2 = np.array((x2, y2))
        euclid_dist = np.sqrt(np.dot((p1 - p2).T, (p1 - p2)))
        return euclid_dist < threshold

    def evaluate_metrics(self):
        cnt_match = 0
        cnt_no_match = 0
        cnt_no_pred = 0
        cnt_no_frame = 0

        for i in range(self.frames_idx[0], self.frames_idx[1] + 1):
            if i not in self.gt_data_points:
                cnt_no_frame += 1
                continue
            if i not in self.pred_points:
                cnt_no_pred += 1
                continue

            p1 = self.gt_data_points[i]
            p2 = self.pred_points[i]

            if self.is_match(*p1['points'], p2['x'], p2['y']):
                cnt_match += 1
            else:
                cnt_no_match += 1

        total_frames = len(self.gt_data_points)
        print(f'Total frames: {total_frames}')
        print(f'Total predictions: {len(self.pred_points)}')
        print(f'Matches: {cnt_match} ({cnt_match / total_frames:.3f})')
        print(f'No matches: {cnt_no_match} ({cnt_no_match / total_frames:.3f})')
        print(f'No predictions: {cnt_no_pred} ({cnt_no_pred / total_frames:.3f})')
        print(f'No frame data: {cnt_no_frame} ({cnt_no_frame / (self.frames_idx[1] - self.frames_idx[0] + 1):.3f})')

    def compute_tracking_sequence(self):
        norm_width = 1920
        norm_height = 1080

        gt_seq = []
        pred_seq = []

        for i in range(min(self.gt_data_points.keys()), max(self.gt_data_points.keys()) + 1):
            if i in self.gt_data_points:
                if i not in self.pred_points:
                    pred_seq.append((0, 0))
                else:
                    p2 = self.pred_points[i]
                    pred_seq.append((p2['x'] / norm_width, p2['y'] / norm_height))

                x, y = self.gt_data_points[i]['points']
                gt_seq.append((x / norm_width, y / norm_height))

        return gt_seq, pred_seq

    def compute_mse(self):
        gt_seq, pred_seq = self.compute_tracking_sequence()
        mse = mean_squared_error(gt_seq, pred_seq)
        print(f'Mean Squared Error: {mse}')
        return mse

### Preprocessing ed estrazione dei frame dai video

In [ ]:
import os
import xml.etree.ElementTree as ET
import cv2

def convert_xml_and_extract_frames(video_path, xml_file, frames_dir, labels_dir, img_width, img_height, video_name):
    # Parse XML e identifica i frame annotati
    tree = ET.parse(xml_file)
    root = tree.getroot()
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    annotated_frames = {}  # Mappa frame -> annotazioni
    for points in root.findall(".//points"):
        frame = int(points.get("frame"))
        outside = int(points.get("outside"))
        used_in_game = points.find(".//attribute[@name='used_in_game']").text if points.find(".//attribute[@name='used_in_game']") is not None else "1"

        # Ottieni le informazioni di annotazione per ogni frame
        x, y = map(float, points.get("points").split(","))

        # Se "used_in_game" è uguale a 0, il frame deve essere ignorato, quindi non ha annotazioni
        if used_in_game == "0":
            annotated_frames[frame] = ""  # File vuoto
        else:
            # Normalizzazione delle coordinate
            x_center = x / img_width
            y_center = y / img_height
            width = 0.05  # Approssimazione della dimensione
            height = 0.05
            # Se il frame è visibile (outside=0), crea le annotazioni
            if outside == 0:
                annotated_frames[frame] = f"0 {x_center} {y_center} {width} {height}\n"
            else:
                # Se fuori dal campo (outside=1), puoi scrivere un file txt vuoto o con una convenzione YOLO
                annotated_frames[frame] = ""  # File vuoto o con un formato che YOLO comprende come "assenza di oggetti"

    # Estrai i frame dal video e genera i file .txt
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Crea il nome del frame
        frame_name = f"{video_name}_frame{frame_count:06d}.jpg"  # Salva come .jpg
        frame_file_path = os.path.join(frames_dir, frame_name)

        # Salva il frame come immagine JPEG con qualità 90
        cv2.imwrite(frame_file_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

        # Crea il file .txt (anche per i frame senza annotazioni)
        txt_file_path = os.path.join(labels_dir, f"{video_name}_frame{frame_count:06d}.txt")
        with open(txt_file_path, "w") as f:
            # Se ci sono annotazioni, scrivile nel file, altrimenti crea un file vuoto
            annotation = annotated_frames.get(frame_count, "")
            if annotation == "":  # Se non ci sono annotazioni o se used_in_game=0
                # In YOLO, un file vuoto significa "assenza di oggetti"
                f.write("")  # Scrive un file vuoto o puoi scrivere una convenzione YOLO (ad esempio '0 0 0 0 0')
            else:
                f.write(annotation)  # Scrive la label (con annotazioni)

        frame_count += 1

    cap.release()



In [ ]:
# Creazione di una cartella unica per tutti i frame e le etichette
frames_output_dir = "/content/drive/MyDrive/dataset/train/images"
labels_output_dir = "/content/drive/MyDrive/dataset/train/labels"

# Esegui per ciascun video e XML nella cartella dataset su Google Drive
videos = [
    ("ID-1", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-1.avi", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-1.xml"),
    ("ID-2", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-2.avi", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-2.xml"),
    ("ID-3", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-3.avi", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-3.xml"),
    ("ID-4", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-4.avi", "/content/drive/MyDrive/dataset/dataset_soccer/train/ID-4.xml")
]

for video_name, video_path, xml_file in videos:
    convert_xml_and_extract_frames(
        video_path=video_path,
        xml_file=xml_file,
        frames_dir=frames_output_dir,
        labels_dir=labels_output_dir,
        img_width=1920,
        img_height=1088,
        video_name=video_name
    )

print("Tutti i frame e le etichette sono stati salvati nelle rispettive cartelle su Google Drive.")

In [ ]:
# Creazione di una cartella unica per tutti i frame e le etichette
frames_output_dir = "/content/drive/MyDrive/dataset/test/images"
labels_output_dir = "/content/drive/MyDrive/dataset/test/labels"

# Esegui per ciascun video e XML nella cartella dataset su Google Drive
videos = [
    ("ID-5", "/content/drive/MyDrive/dataset/dataset_soccer/test/ID-5.avi", "/content/drive/MyDrive/dataset/dataset_soccer/test/ID-5.xml"),
    ("ID-6", "/content/drive/MyDrive/dataset/dataset_soccer/test/ID-6.avi", "/content/drive/MyDrive/dataset/dataset_soccer/test/ID-6.xml"),

]

for video_name, video_path, xml_file in videos:
    convert_xml_and_extract_frames(
        video_path=video_path,
        xml_file=xml_file,
        frames_dir=frames_output_dir,
        labels_dir=labels_output_dir,
        img_width=1920,
        img_height=1088,
        video_name=video_name
    )

print("Tutti i frame e le etichette sono stati salvati nelle rispettive cartelle su Google Drive.")

Per le operazioni di preparazione del dataset, è stato utilizzato il tool online **Roboflow**, che ha svolto un ruolo fondamentale nel migliorare la qualità e la quantità dei dati. In particolare, grazie alle sue funzionalità di **data augmentation**, è stato possibile ottenere una **triplicazione degli esempi positivi**, generando varianti delle immagini originali con trasformazioni come rotazioni, ridimensionamenti e modifiche di luminosità. Questo ha consentito di migliorare significativamente la capacità del modello di generalizzare a situazioni diverse.

Le principali operazioni sono state:
1. Flip: Horizontal & Vertical
2. Random brigthness [-15%,+15%]
3. Blur: fino a 1.4px

![](https://cdn.prod.website-files.com/5f6bc60e665f54545a1e52a5/611410d27864224708a6502f_logo.webp)

# ADDESTRAMENTO DEL MODELLO

# MODEL

Per il nostro progetto, abbiamo scelto YOLO (You Only Look Once) come modello di riferimento per la rilevazione degli oggetti. La decisione è stata motivata da diverse considerazioni che riguardano le esigenze specifiche del nostro scenario e le caratteristiche tecniche di YOLO

In [ ]:
model=YOLO('yolo11n.pt')


Per il nostro progetto, abbiamo addestrato un primo modello YOLO utilizzando il nostro dataset personalizzato, arricchito con tecniche di data augmentation per migliorare la robustezza e la generalizzazione del modello. Di seguito sono riportati i dettagli principali della fase di addestramento

In [ ]:
model=YOLO('yolo11n.pt')
model.train(
    data='/content/drive/MyDrive/model/yolo_output/training_yolo.yaml',  # File YAML del dataset
    epochs=30,  # Numero di epoche
    batch=16,    # Dimensione del batch
    imgsz=1024,  # Dimensione dell'immagine
    patience=4, #Early stop dopo 4 epoche
    conf=0.2, #Abbassiamo la confidenza di predizione a 0.2
    warmup_epochs=3,
    lr0=0.0005, # Learning rate iniziale (più basso per una convergenza più stabile)
    lrf=0.01,   # Learning rate finale
    save=True,     # Salva i pesi
    save_period=5, # Salva ogni 5 epoche
    project='/content/drive/MyDrive/model/yolo_output',  # Salvataggio modello
    device='cuda', # Usa la GPU
    workers=4,         # Numero di workers per il caricamento
    verbose=True
)

#Calcolo delle predizioni in un file json

Ottenuto il nostro modello, procediamo con le predizioni

## Costruzione Json unico con predizioni di entrambi i video

In [ ]:
import os
import json
from ultralytics import YOLO
import numpy as np  # Per la gestione degli array

model_path = "/content/drive/MyDrive/model/yolo_output/train/weights/best.pt"  # O il percorso del modello che vuoi caricare

# Carica il modello
model = YOLO(model_path)

# Percorso alla cartella con i frame di test
test_frames_dir = '/content/drive/MyDrive/dataset/test/images'

# Crea una lista dei file immagine (frame di test)
test_images = [os.path.join(test_frames_dir, f) for f in os.listdir(test_frames_dir) if f.endswith(('.jpg', '.png'))]

# Dizionario per memorizzare le predizioni in formato JSON
predictions = {}

# Esegui le predizioni per ogni immagine di test
for image_path in test_images:
    # Ottieni il nome del file (ad esempio, "ID-6_frame002000.jpg")
    file_name = os.path.basename(image_path)

    # Esegui la predizione sul frame
    results = model.predict(image_path,imgsz=1024)

    # Controlla se ci sono predizioni (bounding boxes)
    if len(results[0].boxes) > 0:  # Se ci sono delle predizioni (risultati)
        boxes = results[0].boxes.xywh.tolist()  # Bounding boxes
        classes = results[0].boxes.cls.tolist()  # Classi delle predizioni
        names = results[0].names  # Nomi delle classi
        confidences = results[0].boxes.conf.tolist()  # Confidenze delle predizioni

        # Iterate attraverso i risultati
        for box, cls, conf in zip(boxes, classes, confidences):
            x_center, y_center, width, height = box  # Coordinate del bounding box
            confidence = conf
            detected_class = cls
            name = names[int(cls)]

            # Calcola la posizione del centro del bounding box
            x_center_pixel = x_center  
            y_center_pixel = y_center  

            # Aggiungi il file e la predizione al dizionario
            predictions[file_name] = {"x": float(x_center_pixel), "y": float(y_center_pixel)}  # Converte in float per serializzazione

    else: predictions[file_name] = {"x": -1, "y": -1}

# Salva le predizioni in un file JSON
predictions_file = '/content/drive/MyDrive/model/yolo_output/predictions_final.json'
with open(predictions_file, 'w') as f:
    json.dump(predictions, f, indent=4)

print(f"Le predizioni sono state salvate in: {predictions_file}")


In [ ]:
import json

# Percorso del file JSON originale
pred_file = '/content/drive/MyDrive/model/yolo_output/predictions_final.json'

# Carica il file JSON delle predizioni
with open(pred_file, 'r') as f:
    predictions = json.load(f)

# Crea due dizionari per separare le predizioni per il video 5 e video 6
predictions_video_5 = {}
predictions_video_6 = {}

# Loop attraverso tutte le predizioni
for frame_name, coords in predictions.items():
    if "ID-5" in frame_name:  # Verifica se il frame appartiene al video 5
        predictions_video_5[frame_name] = coords
    elif "ID-6" in frame_name:  # Verifica se il frame appartiene al video 6
        predictions_video_6[frame_name] = coords

# Salva le predizioni per il video 5 in un nuovo file JSON
video_5_pred_file = '/content/drive/MyDrive/model/yolo_output/predictions_video_5_final.json'
with open(video_5_pred_file, 'w') as f:
    json.dump(predictions_video_5, f, indent=4)

# Salva le predizioni per il video 6 in un nuovo file JSON
video_6_pred_file = '/content/drive/MyDrive/model/yolo_output/predictions_video_6_final.json'
with open(video_6_pred_file, 'w') as f:
    json.dump(predictions_video_6, f, indent=4)

print(f"Predizioni video 5 salvate in: {video_5_pred_file}")
print(f"Predizioni video 6 salvate in: {video_6_pred_file}")

### Creazione Json per video 5

In [ ]:
import json

# Percorso del file JSON per le predizioni del video 5
pred_file = '/content/drive/MyDrive/model/yolo_output/predictions_video_5_final.json'

# Carica il file JSON delle predizioni
with open(pred_file, 'r') as f:
    predictions = json.load(f)

# Crea un nuovo dizionario con i nomi modificati
modified_predictions = {}

# Modifica il nome del frame rimuovendo la parte 'ID-5_frame' e lascia solo il numero del frame
for frame_name, coords in predictions.items():
    # Estrai la parte numerica del nome del file (rimuovi "ID-5_frame" se presente)
    new_frame_name = frame_name.replace("ID-5_frame", "")

    # Aggiungi al dizionario modificato con il nuovo nome del frame
    modified_predictions[new_frame_name] = coords

# Salva le predizioni con i nuovi nomi in un nuovo file JSON
modified_pred_file = '/content/drive/MyDrive/model/yolo_output/modified_predictions_video_5.json'
with open(modified_pred_file, 'w') as f:
    json.dump(modified_predictions, f, indent=4)

print(f"Le predizioni modificate sono state salvate in: {modified_pred_file}")


### Creazione Json per video 6

In [ ]:
import json

# Percorso del file JSON per le predizioni del video 5
pred_file = '/content/drive/MyDrive/model/yolo_output/predictions_video_6_final.json'

# Carica il file JSON delle predizioni
with open(pred_file, 'r') as f:
    predictions = json.load(f)

# Crea un nuovo dizionario con i nomi modificati
modified_predictions = {}

# Modifica il nome del frame rimuovendo la parte 'ID-5_frame' e lascia solo il numero del frame
for frame_name, coords in predictions.items():
    # Estrai la parte numerica del nome del file (rimuovi "ID-5_frame" se presente)
    new_frame_name = frame_name.replace("ID-6_frame", "")

    # Aggiungi al dizionario modificato con il nuovo nome del frame
    modified_predictions[new_frame_name] = coords

# Salva le predizioni con i nuovi nomi in un nuovo file JSON
modified_pred_file = '/content/drive/MyDrive/model/yolo_output/modified_predictions_video_6.json'
with open(modified_pred_file, 'w') as f:
    json.dump(modified_predictions, f, indent=4)

print(f"Le predizioni modificate sono state salvate in: {modified_pred_file}")

Andiamo a fare una prima valutazione delle metriche obiettivo dopo assicurati degli altri parametri come precision,recall,loss.

In [ ]:
# Percorsi ai tuoi file
gt_ann_file = '/content/drive/MyDrive/dataset/dataset_soccer/test/ID-5.xml'  # Ground truth file
pred_file = '/content/drive/MyDrive/model/yolo_output/modified_predictions_video_5.json'  # Predizioni in formato JSON

# Creazione dell'oggetto evaluator
evaluator = TrackingEvaluator(gt_ann_file, pred_file)

# Caricamento dei dati
evaluator.load_data()

# Calcolare gli indici dei frame
evaluator.compute_frame_indices()

# Valutazione delle metriche
evaluator.evaluate_metrics()

# Calcolare MSE
mse1 = evaluator.compute_mse()

In [ ]:
# Percorsi ai tuoi file
gt_ann_file = '/content/drive/MyDrive/dataset/dataset_soccer/test/ID-6.xml'  # Ground truth file
pred_file = '/content/drive/MyDrive/model/yolo_output/modified_predictions_video_6.json'  # Predizioni in formato JSON

# Creazione dell'oggetto evaluator
evaluator = TrackingEvaluator(gt_ann_file, pred_file)

# Caricamento dei dati
evaluator.load_data()

# Calcolare gli indici dei frame
evaluator.compute_frame_indices()

# Valutazione delle metriche
evaluator.evaluate_metrics()

# Calcolare MSE
mse2 = evaluator.compute_mse()

In [ ]:
media=(mse1+mse2)/2
print("MSE medio calcolato =",media)

Come possiamo notare abbiamo ottenuto dei primi risultati che mostrano un modello buono ma che ancora presenta punti di debolezza in situazione di occlusione e bordi, tuttavia attraverso l'attività di post processing si possono migliorare ulteriormente le predizioni.


Ordiniamo le predizioni e procediamo con le operazioni di post processing

In [ ]:
import json
# Percorso del file JSON di input e output
input_file = '/content/drive/MyDrive/model/yolo_output/modified_predictions_video_5.json'
output_file = '/content/drive/MyDrive/model/yolo_output/video5_ordinato.json'

# Ordina i frame per nome (numerico) per garantire l'ordine corretto
def extract_frame_number(frame_name):
    """Estrai il numero del frame dalla stringa del nome del file."""
    return int(frame_name.split('_')[-1].replace('.jpg', '').replace('.png', '').replace('frame', ''))



# Carica il file JSON di input
with open(input_file, 'r') as f:
    predictions = json.load(f)

sorted_predictions = dict(sorted(predictions.items(), key=lambda item: extract_frame_number(item[0])))

# Salva il file JSON con i frame estesi
with open(output_file, 'w') as f:
    json.dump(sorted_predictions, f, indent=4)

print(f"Predizioni estese salvate in: {output_file}")

# Post Processing

## 1 Fase: Eliminiamo i falsi positivi
In una prima fase andiamo ad eliminare le predizioni isolate , ovvero le predizioni che erroneamente il modello ha predetto.
utilizziamo la logica di fare detection di alcuni punti di cumulo isolati e statici.

Il pallone,infatti, è solitamente visibile in aree di gioco dinamiche, dove ci sono interazioni tra i giocatori e il movimento del gioco. Predizioni isolate in punti del campo poco utilizzati (es. centro fisso del frame o angoli statici) non sono coerenti con le dinamiche del gioco.
Inoltre, il pallone non è presente continuamente nel campo visivo, ad esempio, quando viene calciato fuori o è coperto da altri oggetti.

Predizioni isolate possono rappresentare:
* Rilevamenti errati dovuti a elementi visivamente simili al pallone (es. linee bianche, luci riflettenti).
* Errori sistematici del modello in aree poco rappresentative del dataset di training.

Queste predizioni tendono a creare rumore nei risultati, riducendo la precisione complessiva del modello.

In [ ]:
import json
import math

# Percorso del file JSON di input e output
input_file = '/content/drive/MyDrive/model/yolo_output/video5_ordinato.json'
output_file = '/content/drive/MyDrive/model/yolo_output/video5_SenzaOutlier.json'

# Carica il file JSON di input
with open(input_file, 'r') as f:
    predictions = json.load(f)

# Converte le predizioni in una lista per iterazione
frames = list(predictions.keys())
values = list(predictions.values())

# Parametri
window_size = 40  # Dimensione della finestra
static_threshold = 5  # Soglia massima di differenza per considerare i frame "statici"
min_neighbors = 5  # Numero minimo di frame -1, -1 richiesti prima e dopo il blocco

# Funzione per calcolare se un blocco è statico
def is_static_block(values, start, end, threshold):
    """Verifica se il blocco è statico."""
    for i in range(start, end - 1):
        if values[i]["x"] != -1 and values[i]["y"] != -1 and values[i + 1]["x"] != -1 and values[i + 1]["y"] != -1:
            dx = abs(values[i]["x"] - values[i + 1]["x"])
            dy = abs(values[i]["y"] - values[i + 1]["y"])
            if dx > threshold or dy > threshold:
                return False
    return True

# Funzione per controllare se ci sono abbastanza frame -1, -1 prima e dopo il blocco
def has_null_neighbors(values, start, end, min_neighbors):
    """Verifica se ci sono almeno `min_neighbors` frame -1, -1 prima e dopo il blocco."""
    before = all(values[max(0, start - i)]["x"] == -1 and values[max(0, start - i)]["y"] == -1 for i in range(1, min_neighbors + 1))
    after = all(values[min(len(values) - 1, end + i)]["x"] == -1 and values[min(len(values) - 1, end + i)]["y"] == -1 for i in range(1, min_neighbors + 1))
    return before and after

# Depura il file JSON
cleaned_predictions = predictions.copy()

for i in range(len(values) - window_size + 1):
    # Definisci la finestra
    start = i
    end = i + window_size

    # Trova il primo e l'ultimo frame validi nella finestra
    valid_frames = [j for j in range(start, end) if values[j]["x"] != -1 and values[j]["y"] != -1]

    if len(valid_frames) > 2:  # Deve esserci più di 2 frame validi
        first_valid = valid_frames[0]
        last_valid = valid_frames[-1]

        # Controlla se il blocco è statico e se ha abbastanza frame -1, -1 prima e dopo
        if is_static_block(values, first_valid, last_valid, static_threshold) and has_null_neighbors(values, first_valid, last_valid, min_neighbors):
            # Setta tutti i frame della finestra a -1, -1
            for j in range(start, end):
                cleaned_predictions[frames[j]] = {"x": -1, "y": -1}
            #print(f"Blocco da {frames[start]} a {frames[end-1]} impostato a -1, -1.")



# Salva il file JSON depurato
with open(output_file, 'w') as f:
    json.dump(cleaned_predictions, f, indent=4)

print(f"Predizioni depurate salvate in: {output_file}")


## Riduzione falsi negativi

### Metodo di Interpolazione:

Per ridurre le situazioni in cui il modello riscontra difficoltà nella tracking della palla si utilizza il metodo dell'**interpolazione lineare**, che calcola le coordinate intermedie tra:
Ultimo frame prima del buco.
Primo frame dopo il buco.
La traiettoria viene ricostruita dividendo lo spostamento totale tra le due posizioni in intervalli uguali per ogni frame mancante.
Limite di 20 Frame:

Limitiamo l'interpolazione a 20 frame consecutivi per garantire che le predizioni siano ancora realistiche:
Per buchi più lunghi, è probabile che il pallone abbia cambiato direzione o sia stato influenzato da eventi esterni, rendendo l'interpolazione meno affidabile.

In [ ]:
import json

# Percorso del file JSON di input e output
input_file = '/content/drive/MyDrive/model/yolo_output/video6_SenzaOutlier.json'
output_file = '/content/drive/MyDrive/model/yolo_output/video6_Interpolato.json'

# Carica il file JSON di input
with open(input_file, 'r') as f:
    predictions = json.load(f)

# Converte le predizioni in una lista per iterazione
frames = list(predictions.keys())
values = list(predictions.values())

# Numero massimo di buchi da interpolare
max_gap = 20

# Funzione per interpolare
def interpolate_frames(start_coords, end_coords, gap_length):
    """Interpolazione lineare tra due estremi."""
    interpolated = []
    x_step = (end_coords["x"] - start_coords["x"]) / (gap_length + 1)
    y_step = (end_coords["y"] - start_coords["y"]) / (gap_length + 1)
    for i in range(1, gap_length + 1):
        interpolated.append({
            "x": start_coords["x"] + i * x_step,
            "y": start_coords["y"] + i * y_step
        })
    return interpolated

# Itera sui frame per identificare e riempire i buchi
cleaned_predictions = predictions.copy()
i = 0

while i < len(values):
    # Trova l'inizio del buco
    if values[i]["x"] != -1 and values[i]["y"] != -1:
        start_index = i
        # Cerca l'inizio del buco
        i += 1
        while i < len(values) and values[i]["x"] == -1 and values[i]["y"] == -1:
            i += 1
        end_index = i

        # Se il buco è valido e non troppo grande
        gap_length = end_index - start_index - 1
        if gap_length > 0 and gap_length <= max_gap and end_index < len(values):
            # Ottieni le coordinate degli estremi
            start_coords = values[start_index]
            end_coords = values[end_index]

            # Interpola i frame mancanti
            interpolated = interpolate_frames(start_coords, end_coords, gap_length)

            # Aggiorna i frame mancanti con le coordinate interpolate
            for j in range(gap_length):
                frame_index = start_index + j + 1
                cleaned_predictions[frames[frame_index]] = interpolated[j]
                print(f"Frame {frames[frame_index]} interpolato: {interpolated[j]}")
    else:
        i += 1

# Salva il file JSON con i frame interpolati
with open(output_file, 'w') as f:
    json.dump(cleaned_predictions, f, indent=4)

print(f"Predizioni interpolate salvate in: {output_file}")


### Edge Handling
Infine la tecnica di edge handling viene utilizzata per gestire predizioni che terminano in una posizione vicina al bordo dell'immagine, ma non direttamente sul bordo. Questo comportamento spesso indica che il pallone sta uscendo dal campo visivo, ma la predizione si interrompe prematuramente.

* In un contesto realistico, il pallone si muove frequentemente verso i bordi del frame, ad esempio durante tiri, passaggi lunghi o situazioni di gioco veloci.
* Utilizzando la velocità e la direzione calcolate dai frame precedenti, estendiamo la predizione in modo graduale fino a quando il pallone raggiunge il bordo, migliorando così la continuità del tracciamento.




In [ ]:
import json

# Percorso del file JSON di input e output
input_file = '/content/drive/MyDrive/model/yolo_output/video6_Interpolato.json'
output_file = '/content/drive/MyDrive/model/yolo_output/video6_252320.json'

# Carica il file JSON di input
with open(input_file, 'r') as f:
    predictions = json.load(f)

# Dimensioni dell'immagine
img_width = 1920
img_height = 1080

# Converte le predizioni in una lista per iterazione
frames = list(predictions.keys())
values = list(predictions.values())

# Funzione per verificare se una posizione è fuori dai bordi
def is_out_of_bounds(x, y, img_width, img_height):
    return x < 0 or x > img_width or y < 0 or y > img_height

# Itera sui frame per estendere i blocchi validi
cleaned_predictions = predictions.copy()
i = 0

while i < len(values):
    # Trova il blocco di frame validi
    if values[i]["x"] != -1 and values[i]["y"] != -1:
        start_index = i
        while i < len(values) and values[i]["x"] != -1 and values[i]["y"] != -1:
            i += 1
        end_index = i - 1

        # Verifica se il blocco termina senza essere ai bordi
        last_coords = values[end_index]
        if not is_out_of_bounds(last_coords["x"], last_coords["y"], img_width, img_height):
            # Calcola lo scarto usando gli ultimi due frame validi
            if end_index > start_index:  # Assicuriamoci che ci siano almeno due frame validi
                second_last_coords = values[end_index - 1]
                delta_x = last_coords["x"] - second_last_coords["x"]
                delta_y = last_coords["y"] - second_last_coords["y"]

                # Estendi i frame successivi finché non si raggiunge un bordo
                current_x = last_coords["x"]
                current_y = last_coords["y"]
                frame_index = end_index + 1

                while frame_index < len(values):
                    current_x += delta_x
                    current_y += delta_y

                    # Se la palla esce dai bordi, interrompi
                    if is_out_of_bounds(current_x, current_y, img_width, img_height):
                        break

                    # Aggiorna il frame con le nuove coordinate
                    cleaned_predictions[frames[frame_index]] = {"x": current_x, "y": current_y}
                    print(f"Frame {frames[frame_index]} esteso: ({current_x}, {current_y})")

                    frame_index += 1
    else:
        i += 1

# Salva il file JSON con i frame estesi
with open(output_file, 'w') as f:
    json.dump(cleaned_predictions, f, indent=4)

print(f"Predizioni estese salvate in: {output_file}")


# Valutiamo con classe Tracking Evaluator

Dopo aver applicato le tecniche di **post-processing**, rivalutiamo le metriche del modello per verificare i miglioramenti ottenuti. Ci aspettiamo un aumento significativo delle prestazioni, con una riduzione dei falsi positivi e una gestione più accurata dei falsi negativi. In particolare, l'**interpolazione lineare** e l'**edge handling** hanno reso i movimenti del pallone più naturali e fluidi, migliorando la coerenza del tracciamento. Questi interventi hanno contribuito a un notevole incremento nell'accuratezza complessiva delle predizioni, rendendo il modello più robusto e affidabile.

## MSE per Video 5

In [ ]:
# Percorsi ai tuoi file
gt_ann_file = '/content/drive/MyDrive/dataset/dataset_soccer/test/ID-5.xml'  # Ground truth file
pred_file = '/content/drive/MyDrive/model/yolo_output/video5_252320.json'  # Predizioni in formato JSON

# Creazione dell'oggetto evaluator
evaluator = TrackingEvaluator(gt_ann_file, pred_file)

# Caricamento dei dati
evaluator.load_data()

# Calcolare gli indici dei frame
evaluator.compute_frame_indices()

# Valutazione delle metriche
evaluator.evaluate_metrics()

# Calcolare MSE
mse1 = evaluator.compute_mse()


##MSE per video 6

In [ ]:
# Percorsi ai tuoi file
gt_ann_file = '/content/drive/MyDrive/dataset/dataset_soccer/test/ID-6.xml'  # Ground truth file
pred_file = '/content/drive/MyDrive/model/yolo_output/video6_252320.json'  # Predizioni in formato JSON

# Creazione dell'oggetto evaluator
evaluator = TrackingEvaluator(gt_ann_file, pred_file)

# Caricamento dei dati
evaluator.load_data()

# Calcolare gli indici dei frame
evaluator.compute_frame_indices()

# Valutazione delle metriche
evaluator.evaluate_metrics()

# Calcolare MSE
mse2 = evaluator.compute_mse()


In [ ]:
media=(mse1+mse2)/2
print("Risultato ottenuto:",media)


# **Metriche Obiettivo Finali**


In [ ]:
from IPython.display import display, Markdown

# Crea una tabella markdown per mostrare le metriche
table_md = """
# **Metriche Obiettivo Finali**

| **Metrica**    | **VIDEO 5** | **VIDEO 6** | **MEDIA**     |
|----------------|-------------|-------------|---------------|
| **MSE**        | 0.0384      | 0.00225     | 0.020332      |
"""

# Mostra la tabella in Colab
display(Markdown(table_md))


#Generazione dei video con traiettoria

In [ ]:
import cv2
import os
import json
import math

def draw_bounding_boxes_with_trajectory(image_path, prediction, trajectory, max_frames, max_distance=100, color=(0, 0, 255)):
    """
    Disegna i bounding box e la traiettoria su un'immagine usando le predizioni JSON,
    mantenendo solo una traccia limitata nel tempo e smorzando variazioni eccessive.

    :param image_path: Percorso dell'immagine.
    :param prediction: Predizione (dizionario con 'x' e 'y').
    :param trajectory: Lista delle coordinate precedenti per la traiettoria.
    :param max_frames: Numero massimo di frame da mantenere nella traiettoria.
    :param max_distance: Distanza massima consentita tra due punti consecutivi per la traiettoria.
    :param color: Colore del bounding box e della traiettoria.
    :return: Immagine con bounding box e traiettoria sovrapposti.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Immagine non trovata: {image_path}")
        return None

    height, width, _ = image.shape
    current_position = None

    if prediction["x"] != -1 and prediction["y"] != -1:
        x = prediction["x"]
        y = prediction["y"]
        box_size = 50  # Dimensione del bounding box in pixel
        x1 = int(x - box_size / 2)
        y1 = int(y - box_size / 2)
        x2 = int(x + box_size / 2)
        y2 = int(y + box_size / 2)

        # Salva la posizione corrente per la traiettoria
        current_position = (x, y)

        # Disegna il bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, "Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Controllo variazione eccessiva
    if trajectory and current_position:
        last_position = trajectory[-1]
        if last_position is not None:
            distance = math.sqrt((current_position[0] - last_position[0]) ** 2 +
                                 (current_position[1] - last_position[1]) ** 2)
            if distance > max_distance:
                print(f"Variazione eccessiva ignorata: {distance:.2f} pixel")
                current_position = None

    # Disegna la traiettoria
    if trajectory and len(trajectory) > 1:
        for i in range(1, len(trajectory)):
            if trajectory[i - 1] is not None and trajectory[i] is not None:
                pt1 = tuple(map(int, trajectory[i - 1]))
                pt2 = tuple(map(int, trajectory[i]))
                cv2.line(image, pt1, pt2, color, 2)


    # Aggiungi la posizione corrente alla traiettoria
    if current_position:
        trajectory.append(current_position)
    else:
        trajectory.append(None)  # Interrompe la traiettoria se la palla non è visibile

    if len(trajectory) > max_frames:
        trajectory.pop(0)  # Rimuovi i punti più vecchi

    return image

def create_video_with_bboxes_and_trajectory_json(image_dir, json_file, output_video_path, fps=25, max_trajectory_seconds=0.5):
    """
    Crea un video con bounding box e traiettoria sovrapposti usando predizioni da file JSON.

    :param image_dir: Directory contenente le immagini.
    :param json_file: File JSON contenente le predizioni.
    :param output_video_path: Percorso per salvare il video generato.
    :param fps: Frame per secondo del video.
    :param max_trajectory_seconds: Numero massimo di secondi da mantenere nella traiettoria.
    """
    # Carica le predizioni
    with open(json_file, "r") as f:
        predictions = json.load(f)

    # Ottieni la lista delle immagini
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])

    if not image_files:
        print(f"Nessuna immagine trovata nella directory: {image_dir}")
        return

    # Leggi la prima immagine per ottenere dimensioni
    first_image_path = os.path.join(image_dir, image_files[0])
    first_image = cv2.imread(first_image_path)
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec per MP4
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    trajectory = []  # Lista per salvare la traiettoria
    max_frames = int(max_trajectory_seconds * fps)  # Numero massimo di frame nella traiettoria

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        frame_number = image_file.split("_frame")[1].split(".")[0]  # Es. "000000"
        json_key = f"{frame_number}.jpg"  # Es. "000000.jpg"

        # Ottieni la predizione dal file JSON
        prediction = predictions.get(json_key, {"x": -1, "y": -1})

        # Disegna bounding box e traiettoria
        annotated_image = draw_bounding_boxes_with_trajectory(image_path, prediction, trajectory, max_frames)
        if annotated_image is not None:
            video_writer.write(annotated_image)

    video_writer.release()
    print(f"Video generato con successo: {output_video_path}")

# Percorsi per il video 6
image_dir = "/content/drive/MyDrive/dataset/test/video5"  # Directory immagini di test
json_file = "/content/drive/MyDrive/model/yolo_output/video5_252320.json"  # Predizioni per video 6
output_video = "/content/drive/MyDrive/model/yolo_output/videoAnnotato5.mp4"  # Video di output per video 6

# Crea il video
create_video_with_bboxes_and_trajectory_json(image_dir, json_file, output_video, fps=25, max_trajectory_seconds=0.5)


In [ ]:
import cv2
import os
import json
import math

def draw_bounding_boxes_with_trajectory(image_path, prediction, trajectory, max_frames, max_distance=100, color=(0, 0, 255)):
    """
    Disegna i bounding box e la traiettoria su un'immagine usando le predizioni JSON,
    mantenendo solo una traccia limitata nel tempo e smorzando variazioni eccessive.

    :param image_path: Percorso dell'immagine.
    :param prediction: Predizione (dizionario con 'x' e 'y').
    :param trajectory: Lista delle coordinate precedenti per la traiettoria.
    :param max_frames: Numero massimo di frame da mantenere nella traiettoria.
    :param max_distance: Distanza massima consentita tra due punti consecutivi per la traiettoria.
    :param color: Colore del bounding box e della traiettoria.
    :return: Immagine con bounding box e traiettoria sovrapposti.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Immagine non trovata: {image_path}")
        return None

    height, width, _ = image.shape
    current_position = None

    if prediction["x"] != -1 and prediction["y"] != -1:
        x = prediction["x"]
        y = prediction["y"]
        box_size = 50  # Dimensione del bounding box in pixel
        x1 = int(x - box_size / 2)
        y1 = int(y - box_size / 2)
        x2 = int(x + box_size / 2)
        y2 = int(y + box_size / 2)

        # Salva la posizione corrente per la traiettoria
        current_position = (x, y)

        # Disegna il bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, "Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Controllo variazione eccessiva
    if trajectory and current_position:
        last_position = trajectory[-1]
        if last_position is not None:
            distance = math.sqrt((current_position[0] - last_position[0]) ** 2 +
                                 (current_position[1] - last_position[1]) ** 2)
            if distance > max_distance:
                print(f"Variazione eccessiva ignorata: {distance:.2f} pixel")
                current_position = None

    # Disegna la traiettoria
    if trajectory and len(trajectory) > 1:
        for i in range(1, len(trajectory)):
            if trajectory[i - 1] is not None and trajectory[i] is not None:
                pt1 = tuple(map(int, trajectory[i - 1]))
                pt2 = tuple(map(int, trajectory[i]))
                cv2.line(image, pt1, pt2, color, 2)


    # Aggiungi la posizione corrente alla traiettoria
    if current_position:
        trajectory.append(current_position)
    else:
        trajectory.append(None)  # Interrompe la traiettoria se la palla non è visibile

    if len(trajectory) > max_frames:
        trajectory.pop(0)  # Rimuovi i punti più vecchi

    return image

def create_video_with_bboxes_and_trajectory_json(image_dir, json_file, output_video_path, fps=25, max_trajectory_seconds=0.5):
    """
    Crea un video con bounding box e traiettoria sovrapposti usando predizioni da file JSON.

    :param image_dir: Directory contenente le immagini.
    :param json_file: File JSON contenente le predizioni.
    :param output_video_path: Percorso per salvare il video generato.
    :param fps: Frame per secondo del video.
    :param max_trajectory_seconds: Numero massimo di secondi da mantenere nella traiettoria.
    """
    # Carica le predizioni
    with open(json_file, "r") as f:
        predictions = json.load(f)

    # Ottieni la lista delle immagini
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])

    if not image_files:
        print(f"Nessuna immagine trovata nella directory: {image_dir}")
        return

    # Leggi la prima immagine per ottenere dimensioni
    first_image_path = os.path.join(image_dir, image_files[0])
    first_image = cv2.imread(first_image_path)
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec per MP4
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    trajectory = []  # Lista per salvare la traiettoria
    max_frames = int(max_trajectory_seconds * fps)  # Numero massimo di frame nella traiettoria

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        frame_number = image_file.split("_frame")[1].split(".")[0]  # Es. "000000"
        json_key = f"{frame_number}.jpg"  # Es. "000000.jpg"

        # Ottieni la predizione dal file JSON
        prediction = predictions.get(json_key, {"x": -1, "y": -1})

        # Disegna bounding box e traiettoria
        annotated_image = draw_bounding_boxes_with_trajectory(image_path, prediction, trajectory, max_frames)
        if annotated_image is not None:
            video_writer.write(annotated_image)

    video_writer.release()
    print(f"Video generato con successo: {output_video_path}")

# Percorsi per il video 6
image_dir = "/content/drive/MyDrive/dataset/test/video6"  # Directory immagini di test
json_file = "/content/drive/MyDrive/model/yolo_output/video6_252320.json"  # Predizioni per video 6
output_video = "/content/drive/MyDrive/model/yolo_output/videoAnnotato6.mp4"  # Video di output per video 6

# Crea il video
create_video_with_bboxes_and_trajectory_json(image_dir, json_file, output_video, fps=25, max_trajectory_seconds=0.5)


# Conclusioni
Prospettive future e possibili miglioramenti

* Raccogliere più dati con situazioni diverse (occlusioni, angolazioni, condizioni di illuminazione variabili).
* Aumentare la varietà di esempi per migliorare la generalizzazione del modello.
* Applicare metodi di interpolazione più sofisticati per migliorare la gestione dei falsi negativi.
* Sperimentazione con Modelli Più Complessi(es. YOLO11X)
* Integrazione con Sistemi Avanzati